In [3]:
%matplotlib inline

import time
import math

import numpy as np
import pandas as pd
import datetime as dt
import statistics as stat

from sklearn.ensemble import RandomForestRegressor
from sklearn import cross_validation
from sklearn import preprocessing  # or one-hot-outlier

#Classification Specific:
#from sklearn.metrics import classification_report
#from sklearn.metrics import (brier_score_loss, precision_score, recall_score,
#                             f1_score)
#Regression Specific:
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt

In [4]:
## Load Data Set
df = pd.read_csv('data_sci_snippet.csv')
df.head(1)

,ListingId,LivingArea,NumBedrooms,NumBaths,Pool,ExteriorStories,ListDate,ListPrice,GeoLat,GeoLon,PublicRemarks,CloseDate,ClosePrice,ListingStatus,DwellingType
0,5077399,3054,4,2.5,Private,1,2014-03-04,675000,33.593239,-111.961056,This property OUTSTANDING offers outdoor livin...,NaN,NaN,Cancelled,Single Family - Detached


In [5]:
a = [('a','b','c'),('j','h','g')]
b = ['d','e','f']
c = reduce(lambda a, b: a+b, l)

NameError: name 'reduce' is not defined

In [12]:
import re
from nltk.tokenize import word_tokenize
from nltk.util import ngrams

def get_ngrams(text, n ):
    text = re.sub(r'[\b\(\)\\\"\'\/\[\]\s+\,\.:\?;]', ' ', text)
    text = text.lower()
    n_grams = ngrams(word_tokenize(text), n)
    return [ ' '.join(grams) for grams in n_grams]


unigram = get_ngrams(df['PublicRemarks'][0], 1)
unigram

TypeError: expected string or buffer

In [ ]:
import urllib
from bs4 import BeautifulSoup
import nltk
#from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.collocations import *
import string

url = 'https://en.wikipedia.org/wiki/N-gram'
html = urllib.request.urlopen(url).read()
soup = BeautifulSoup(html)

# kill all script and style elements
for script in soup(["script", "style"]):
    script.extract()    # rip it out

# get text
text = soup.get_text()

# break into lines and remove leading and trailing space on each
lines = (line.strip() for line in text.splitlines())
# break multi-headlines into a line each
chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
# drop blank lines
text = '\n'.join(chunk for chunk in chunks if chunk)

#textFiltered = filter(None,[word.strip(string.punctuation)
#                 for word in text.replace(';','; ').split()
#                 ])

tokenizer = RegexpTokenizer(r'\w+')
tokens = tokenizer.tokenize(text)

finder = TrigramCollocationFinder.from_words(tokens)
trigram_measures = nltk.collocations.TrigramAssocMeasures()
scored = finder.score_ngrams(trigram_measures.raw_freq)
sorted(finder.nbest(trigram_measures.raw_freq, 10))  # Return 10 trigrams

In [1]:
## ++++++  Pipeline ++++++

# Author: Olivier Grisel <olivier.grisel@ensta.org>
#         Peter Prettenhofer <peter.prettenhofer@gmail.com>
#         Mathieu Blondel <mathieu@mblondel.org>
# License: BSD 3 clause
# Source: http://scikit-learn.org/stable/auto_examples/model_selection/grid_search_text_feature_extraction.html

from __future__ import print_function

from pprint import pprint
from time import time
import logging

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline

print(__doc__)

# Display progress logs on stdout
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(levelname)s %(message)s')


###############################################################################
# Load some categories from the training set
categories = [
    'alt.atheism',
    'talk.religion.misc',
]
# Uncomment the following to do the analysis on all the categories
#categories = None

print("Loading 20 newsgroups dataset for categories:")
print(categories)

data = fetch_20newsgroups(subset='train', categories=categories)
print("%d documents" % len(data.filenames))
print("%d categories" % len(data.target_names))
print()

###############################################################################
# define a pipeline combining a text feature extractor with a simple
# classifier
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier()),
])

# uncommenting more parameters will give better exploring power but will
# increase processing time in a combinatorial way
parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),
    #'vect__max_features': (None, 5000, 10000, 50000),
    'vect__ngram_range': ((1, 1), (1, 2), (1, 3)),  # unigrams or bigrams or trigrams
    #'tfidf__use_idf': (True, False),
    #'tfidf__norm': ('l1', 'l2'),
    'clf__alpha': (0.00001, 0.000001),
    'clf__penalty': ('l2', 'elasticnet'),
    #'clf__n_iter': (10, 50, 80),
}

if __name__ == "__main__":
    # multiprocessing requires the fork to happen in a __main__ protected
    # block

    # find the best parameters for both the feature extraction and the
    # classifier
    grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1)

    print("Performing grid search...")
    print("pipeline:", [name for name, _ in pipeline.steps])
    print("parameters:")
    pprint(parameters)
    t0 = time()
    grid_search.fit(data.data, data.target)
    print("done in %0.3fs" % (time() - t0))
    print()

    print("Best score: %0.3f" % grid_search.best_score_)
    print("Best parameters set:")
    best_parameters = grid_search.best_estimator_.get_params()
    for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))

Automatically created module for IPython interactive environment
Loading 20 newsgroups dataset for categories:
['alt.atheism', 'talk.religion.misc']
857 documents
2 categories

Performing grid search...
pipeline: ['vect', 'tfidf', 'clf']
parameters:
{'clf__alpha': (1e-05, 1e-06),
 'clf__penalty': ('l2', 'elasticnet'),
 'vect__max_df': (0.5, 0.75, 1.0),
 'vect__ngram_range': ((1, 1), (1, 2), (1, 3))}
Fitting 3 folds for each of 36 candidates, totalling 108 fits


[Parallel(n_jobs=-1)]: Done   1 jobs       | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done  50 jobs       | elapsed:   34.7s
[Parallel(n_jobs=-1)]: Done 108 out of 108 | elapsed:  1.3min finished


done in 81.137s

Best score: 0.944
Best parameters set:
	clf__alpha: 1e-05
	clf__penalty: 'elasticnet'
	vect__max_df: 1.0
	vect__ngram_range: (1, 3)


### Create a corpus and conduct analysis

In [ ]:
## Concatenate 'PublicRemarks' into a corpus
def concatCol(col):
    text = ''  # Declare text as string
    for n in range(len(col)): # 
        text += str(col[n])+' '
    return text

corpus = concatCol(df['PublicRemarks'])

## Save Text to File
file = open('PublicRemarksCorpus.txt', "w")
file.write(corpus)
file.close()

In [ ]:
# Read 'PublicRemarks' Text from file
textfile = open('PublicRemarksCorpus.txt', 'r') # File
corpus = textfile.read()  # String
#corpus[0:500]

In [ ]:
import re
from nltk.tokenize import word_tokenize
from nltk.util import ngrams

def get_ngrams(text, n ):
    text = re.sub(r'[\b\(\)\\\"\'\/\[\]\s+\,\.:\?;]', ' ', text)
    text = text.lower()
    tokens = word_tokenize(text)
    n_grams = ngrams(tokens, n)
    return [ ' '.join(grams) for grams in n_grams]

unigrams = get_ngrams(corpus, 1)